In [1]:
import pandas as pd
from tqdm import tqdm

In [2]:
# inputs
fn_metadata = '../data/intermediary/element_metadata_15k.jsonl'
fn_real = '../data/intermediary/element_metadata_real_1k.jsonl'
fn_trends = '../data/intermediary/trending_searches_by_category.csv'

In [3]:
df = pd.read_json(fn_metadata, 
                  lines=True)
trends = pd.read_csv(fn_trends)

In [4]:
real = pd.read_json(fn_real, lines=True)

In [5]:
df.head(2)

,text,link,domain,xpath,element_class,category,element,tag,attrs,dimensions,location,area,area_page,fn_input,label,category_standard
0,Why these ads?,None,google.com,/html/body/div[10]/div/div[6]/div/div[1]/div[2...,kxWPvc,ads-general,NaN,div,{'class': ['kxWPvc']},"{'height': 14.0, 'width': 15.0}","{'x': 324, 'y': 185}",210.0,210.0,../data/input/google_search/iPhone-X/2019/11/2...,ads,ads
1,Suggested RefinementsSaleHair Loss Treatments ...,None,google.com,/html/body/div[10]/div/div[6]/div/div[1]/div[2...,rcwta,link-filter_product_refinement,NaN,div,{'data-wta-ad-reasons': 'wtad:CkAIARABGAAgAZoB...,"{'height': 52.0, 'width': 347.0}","{'x': 8, 'y': 224}",18044.0,18044.0,../data/input/google_search/iPhone-X/2019/11/2...,link,google-misc


In [6]:
real.loc[:, 'search'] = real.fn_input.apply(lambda x : ('').join(x.split('/html')[0].split('/')[8:]))

In [41]:
sum([len(row.split('-')) for row in real.search.unique()]) / real.search.nunique()

3.5964360587002098

In [43]:
sum([len(row) for row in real.search.unique()]) / real.search.nunique()

23.19496855345912

In [7]:
df.loc[:, 'search'] = df.fn_input.apply(lambda x : ('').join(x.split('/html')[0].split('/')[8:]))

In [8]:
trends.loc[:, 'search'] = trends['search'].str.replace(' ', '-')

In [9]:
trends.search.nunique()

11648

In [10]:
df['search'].iloc[0]

'Pattern-hair-loss'

In [11]:
cat2searches = {}
for cat in trends.category.unique():
    searches = (trends[trends.category == cat].search.tolist())
    cat2searches[cat] = searches

In [26]:
cat2desc = {
    'b' : 'Business',
    'e' : 'Entertainment',
    'm' : 'Health',
    't' : 'Science and Tech',
    's' : 'Sports',
    'h' : 'Top Stories',
    'real' : 'User Searches',
    'mix' : 'Multiple Categories'
}

In [13]:
import random
from scipy.stats import t

In [33]:
cat2df = {}
for k, v in cat2searches.items():
    _df = df[df.search.isin(v)]
    random.seed(303)
    sample = _df.fn_input.unique()
    random.shuffle(sample)
    sample = sample[:700]

    cat2df[k] = _df[_df.fn_input.isin(sample)]

# _df = df[~df.search.isin(trends.search)]
# random.seed(303)
# sample = _df.fn_input.unique()
# random.shuffle(sample)
# sample = sample[:700]
# cat2df['mix'] = _df[_df.fn_input.isin(sample)]
    
cat2df['All'] = pd.concat(_df for _df in cat2df.values())

_df = real
random.seed(303)
sample = _df.search.unique()
random.shuffle(sample)
sample = sample[:700]
cat2df['real'] = _df[_df.search.isin(sample)]

In [15]:
cat2pixels = {}
for cat, _df in cat2df.items():

    pixels = {
        'area' : [],
        'area_page' : []
    }
    for search, df_ in tqdm(_df.groupby('fn_input')):
        for col in ['area', 'area_page']:
            ad = df_[df_.label == 'ads'][col].sum()
            o = df_[df_.label == 'organic'][col].sum()
            g = df_[(df_.label == 'link') | 
                    (df_.label == 'answer')][col].sum()
            amp = df_[df_.label == 'amp'][col].sum()

            e1 = g / (g + o)
            e2 = (g + ad) / (g + o + ad )
            e3 = (g + amp) / (g + o + amp)
            e4 = (g + amp + ad) / (g + o + amp + ad)
            e5 = g / (g + ad + o + amp)

            e6 =  o / (g + ad + o + amp)


            row = {
                'search_term' : search,
                'ad' : ad,
                'o' : o,
                'g' : g,
                'amp' : amp,
                'e1' : e1,
                'e2' : e2,
                'e3' : e3,
                'e4' : e4,
                'e5' : e5,
                'e6' : e6
            }

            pixels[col].append(row)

    df_pixels = pd.DataFrame(pixels['area'])
    df_pixels_fp = pd.DataFrame(pixels['area_page'])
    
    cat2pixels[cat] = {
        'df_pixels' : df_pixels,
        'df_pixels_fp' : df_pixels_fp
    }

  2%|▏         | 15/700 [00:00<00:04, 149.73it/s]<ipython-input-15-f4de6ff22287>:16: RuntimeWarning: invalid value encountered in double_scalars
  e1 = g / (g + o)
<ipython-input-15-f4de6ff22287>:19: RuntimeWarning: invalid value encountered in double_scalars
  e3 = (g + amp) / (g + o + amp)
 81%|████████  | 566/700 [00:02<00:00, 215.94it/s]<ipython-input-15-f4de6ff22287>:18: RuntimeWarning: invalid value encountered in double_scalars
  e2 = (g + ad) / (g + o + ad )
<ipython-input-15-f4de6ff22287>:20: RuntimeWarning: invalid value encountered in double_scalars
  e4 = (g + amp + ad) / (g + o + amp + ad)
<ipython-input-15-f4de6ff22287>:21: RuntimeWarning: invalid value encountered in double_scalars
  e5 = g / (g + ad + o + amp)
<ipython-input-15-f4de6ff22287>:23: RuntimeWarning: invalid value encountered in double_scalars
  e6 =  o / (g + ad + o + amp)
100%|██████████| 4900/4900 [00:24<00:00, 203.38it/s]


In [16]:
cat2pixels['b']['df_pixels']

,search_term,ad,o,g,amp,e1,e2,e3,e4,e5,e6
0,../data/input/google_search/iPhone-X/2019/11/0...,0.0,208496.000000,1728.000000,0.0,0.008220,0.008220,0.008220,0.008220,0.008220,0.991780
1,../data/input/google_search/iPhone-X/2019/11/0...,0.0,0.000000,178271.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
2,../data/input/google_search/iPhone-X/2019/11/0...,0.0,971.199951,149412.065063,0.0,0.993542,0.993542,0.993542,0.993542,0.993542,0.006458
3,../data/input/google_search/iPhone-X/2019/11/0...,0.0,146383.000000,30960.966980,0.0,0.174581,0.174581,0.174581,0.174581,0.174581,0.825419
4,../data/input/google_search/iPhone-X/2019/11/0...,0.0,17974.199951,112090.484650,0.0,0.861806,0.861806,0.861806,0.861806,0.861806,0.138194
...,...,...,...,...,...,...,...,...,...,...,...
695,../data/input/google_search/iPhone-X/2020/01/0...,0.0,180093.000000,19807.999268,0.0,0.099089,0.099089,0.099089,0.099089,0.099089,0.900911
696,../data/input/google_search/iPhone-X/2020/01/0...,0.0,0.000000,159496.800293,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000
697,../data/input/google_search/iPhone-X/2020/01/0...,0.0,72523.000000,82180.000000,0.0,0.531211,0.531211,0.531211,0.531211,0.531211,0.468789
698,../data/input/google_search/iPhone-X/2020/01/0...,0.0,139145.000000,45562.000000,0.0,0.246672,0.246672,0.246672,0.246672,0.246672,0.753328


In [17]:
def t_student_confidence_interval(data, 
                                  confidence = 0.95,
                                  scale = 100):
    n = len(data)
    m = data.mean()
    std_err = data.sem()
    h = std_err * t.ppf((1 + confidence) / 2, n - 1)
    
    s = f"{scale * m:.1f} ± {h * scale :.1f}"
    return s

In [18]:
index2human = {
    'e1' : 'EQ1: g / (g + o)',
    'e2' : 'EQ2: (g + ad) / (g + o + ad)',
    'e3' : 'EQ3: (g + amp) / (g + o + amp)',
    'e4' : 'EQ4: (g + amp + ad) / (g + o + amp + ad)',
    'e5' : 'EQ5: g / (g + o + ads + amp)',
    
    'e6' : 'EQ6: o / (g + o + ads + amp)',
}

In [19]:
from IPython.display import display, HTML

In [20]:
for cat, v in cat2pixels.items():
    print(cat)
    df_pixels = v['df_pixels']
    results = {}
    for e in [f'e{i}' for i in range(1, 7)]:
        x_goog = t_student_confidence_interval(df_pixels[e])
        x_goog_fp = t_student_confidence_interval(df_pixels_fp[e])
        results[index2human.get(e)] = {
            'above the fold' : x_goog + '%',
            'first page' : x_goog_fp+ '%'
        }
        
    display(HTML(pd.DataFrame(results).T.to_html()))

b


,above the fold,first page
EQ1: g / (g + o),61.7 ± 2.9%,44.5 ± 0.3%
EQ2: (g + ad) / (g + o + ad),66.4 ± 2.7%,45.1 ± 0.3%
EQ3: (g + amp) / (g + o + amp),63.7 ± 2.8%,51.2 ± 0.4%
EQ4: (g + amp + ad) / (g + o + amp + ad),68.3 ± 2.6%,51.7 ± 0.4%
EQ5: g / (g + o + ads + amp),50.6 ± 2.9%,38.1 ± 0.3%
EQ6: o / (g + o + ads + amp),31.7 ± 2.6%,48.3 ± 0.4%


m


,above the fold,first page
EQ1: g / (g + o),80.4 ± 2.1%,44.5 ± 0.3%
EQ2: (g + ad) / (g + o + ad),81.5 ± 2.0%,45.1 ± 0.3%
EQ3: (g + amp) / (g + o + amp),81.1 ± 2.0%,51.2 ± 0.4%
EQ4: (g + amp + ad) / (g + o + amp + ad),82.2 ± 1.9%,51.7 ± 0.4%
EQ5: g / (g + o + ads + amp),74.4 ± 2.4%,38.1 ± 0.3%
EQ6: o / (g + o + ads + amp),17.8 ± 1.9%,48.3 ± 0.4%


e


...

,above the fold,first page
EQ1: g / (g + o),79.4 ± 2.3%,44.5 ± 0.3%
EQ2: (g + ad) / (g + o + ad),79.9 ± 2.3%,45.1 ± 0.3%
EQ3: (g + amp) / (g + o + amp),83.6 ± 2.0%,51.2 ± 0.4%
EQ4: (g + amp + ad) / (g + o + amp + ad),84.0 ± 1.9%,51.7 ± 0.4%
EQ5: g / (g + o + ads + amp),63.3 ± 2.8%,38.1 ± 0.3%
EQ6: o / (g + o + ads + amp),16.0 ± 1.9%,48.3 ± 0.4%


real


,above the fold,first page
EQ1: g / (g + o),47.1 ± 3.0%,44.5 ± 0.3%
EQ2: (g + ad) / (g + o + ad),47.8 ± 3.0%,45.1 ± 0.3%
EQ3: (g + amp) / (g + o + amp),54.7 ± 2.8%,51.2 ± 0.4%
EQ4: (g + amp + ad) / (g + o + amp + ad),55.4 ± 2.7%,51.7 ± 0.4%
EQ5: g / (g + o + ads + amp),40.4 ± 2.8%,38.1 ± 0.3%
EQ6: o / (g + o + ads + amp),44.6 ± 2.7%,48.3 ± 0.4%


All


,above the fold,first page
EQ1: g / (g + o),72.9 ± 1.0%,44.5 ± 0.3%
EQ2: (g + ad) / (g + o + ad),74.6 ± 0.9%,45.1 ± 0.3%
EQ3: (g + amp) / (g + o + amp),76.1 ± 0.9%,51.2 ± 0.4%
EQ4: (g + amp + ad) / (g + o + amp + ad),77.6 ± 0.9%,51.7 ± 0.4%
EQ5: g / (g + o + ads + amp),61.8 ± 1.1%,38.1 ± 0.3%
EQ6: o / (g + o + ads + amp),22.4 ± 0.9%,48.3 ± 0.4%


In [21]:
label2publabel = {
    'link' : 'Google Product',
    'answer' : 'Google Answer',
    'organic' : 'Non-Google',
    'amp' : 'AMP',
    'ads' : 'Ads',
}

In [22]:
labels = [
    'organic',
    'amp',
    'link',
    'answer',
    'ads'
]

In [34]:
for cat, _df in cat2df.items():
    print(f"{cat2desc.get(cat, cat)} (N={_df.fn_input.nunique():,})")
    total_area = _df['area'].sum()
    total_area_fp = _df['area_page'].sum()
    n_pages = _df.fn_input.nunique()
    data = {}
    for label in labels:
        __df = _df[_df.label == label]
        area = (__df['area'].sum() / total_area)
        area_page = (__df['area_page'].sum() / total_area_fp)
        freq = __df[__df['area'] != 0].fn_input.nunique() / n_pages
        freq_page = __df[__df['area_page'] != 0].fn_input.nunique() /n_pages
        data[label2publabel.get(label,label)] = {
            ('Above the fold', 'area') : area,
            ('Above the fold', 'freq') : freq,
            ('First page', 'area') : area_page,
            ('First page', 'freq') : freq_page,
        }

    breakdown = pd.DataFrame(data).T
    breakdown.columns = pd.MultiIndex.from_tuples([   
        ('Above the fold', 'area'),
        ('Above the fold', 'freq'),
        ('First page', 'area'),
        ('First page', 'freq')
    ])
    
    print(((breakdown * 100).round(1).astype(str) + '%'))
    print("*" * 79)

Business (N=700)
               Above the fold        First page        
                         area   freq       area    freq
Non-Google              33.4%  79.0%      49.6%  100.0%
AMP                      4.8%  10.0%       8.8%   64.1%
Google Product          28.4%  84.0%      26.0%  100.0%
Google Answer           18.3%  62.9%      12.9%   99.3%
Ads                     15.1%  18.0%       2.6%   19.3%
*******************************************************************************
Health (N=700)
               Above the fold        First page       
                         area   freq       area   freq
Non-Google              19.0%  77.0%      54.2%  99.9%
AMP                      1.6%   3.6%       5.1%  50.1%
Google Product          28.0%  88.3%      20.5%  99.9%
Google Answer           44.2%  86.0%      19.0%  98.6%
Ads                      7.2%   8.3%       1.2%   8.9%
*******************************************************************************
Entertainment (N=700)
         